In [ ]:
import pandas as pd 
import numpy as np 
import torch
import torch.nn as nn 
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os  

In [4]:
train_dir = "C:\\Users\\Tharusha Kamalanath\\Downloads\\archive(3)\\train" 
test_dir = "C:\\Users\\Tharusha Kamalanath\\Downloads\\archive(3)\\test"

In [5]:
transform = transforms.Compose([transforms.Grayscale(),
                                transforms.Resize((48,48)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,),(0.5,))])

In [ ]:
train_dataset = datasets.ImageFolder(train_dir,transform=transform)
test_dataset = datasets.ImageFolder(test_dir,transform=transform) 

train_loader = DataLoader(train_dataset,batch_size=64,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=64,shuffle=False)



In [8]:
class EmotionCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),                
            
            nn.Conv2d(16, 32, 3, padding=1), 
            nn.ReLU(),
            nn.MaxPool2d(2),                 

            nn.Conv2d(32, 64, 3, padding=1), 
            nn.ReLU(),
            nn.MaxPool2d(2)                  
        )

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 6 * 6, 128),      
            nn.ReLU(),
            nn.Linear(128, 7)               
        )

    def forward(self, x):
        x = self.cnn(x)
        x = self.fc(x)
        return x


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EmotionCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0
    correct = 0
    total = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss:.4f}, Accuracy: {100*correct/total:.2f}%")


In [ ]:
import torchvision
import matplotlib.pyplot as plt

def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(npimg[0], cmap='gray')
    plt.axis('off')
    plt.show()

dataiter = iter(test_loader)
images, labels = next(dataiter)
outputs = model(images.to(device))
_, preds = outputs.max(1)

for i in range(5):
    imshow(images[i])
    print(f"True: {class_names[labels[i]]}, Predicted: {class_names[preds[i]]}")


In [ ]:
pip install opencv-python

In [13]:
torch.save(model.state_dict(), "emotion_model.pth")

In [14]:
import torch
import cv2
import numpy as np
from torchvision import transforms

# Load trained model (adjust path)
model = EmotionCNN()
model.load_state_dict(torch.load("emotion_model.pth", map_location=torch.device('cpu')))
model.eval()

# Class labels
class_names = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']


In [15]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Grayscale(),
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


In [ ]:
cap = cv2.VideoCapture(0)  
ret, frame = cap.read()
cap.release()  

# Convert to grayscale and detect face
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

for (x, y, w, h) in faces:
    roi = gray[y:y+h, x:x+w]
    roi_resized = cv2.resize(roi, (48, 48))
    roi_tensor = transform(roi_resized).unsqueeze(0)

    with torch.no_grad():
        output = model(roi_tensor)
        pred = torch.argmax(output, 1).item()
        label = class_names[pred]

    # Draw label and box
    cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)
    cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

# Convert color (BGR to RGB) for matplotlib
frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

# Display in notebook
plt.figure(figsize=(6,6))
plt.imshow(frame_rgb)
plt.axis('off')
plt.title("Predicted Emotion")
plt.show()
